In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeRegressor
import category_encoders as ce

In [3]:
df = pd.read_csv("/Users/liamsweeney/dat-11-15/Homework/Unit2/data/master.csv", parse_dates = ['visit_date'])

In [5]:
df

,id,visit_date,visitors,calendar_date,day_of_week,holiday,genre,area,latitude,longitude,reserve_visitors
0,air_ba937bf13d40fb24,2016-01-13,25,2016-01-13,Wednesday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN
1,air_ba937bf13d40fb24,2016-01-14,32,2016-01-14,Thursday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN
2,air_ba937bf13d40fb24,2016-01-15,29,2016-01-15,Friday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN
3,air_ba937bf13d40fb24,2016-01-16,22,2016-01-16,Saturday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN
4,air_ba937bf13d40fb24,2016-01-18,6,2016-01-18,Monday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN
...,...,...,...,...,...,...,...,...,...,...,...
252103,air_a17f0778617c76e2,2017-04-21,49,2017-04-21,Friday,0,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,6.0
252104,air_a17f0778617c76e2,2017-04-22,60,2017-04-22,Saturday,0,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,37.0
252105,air_a17f0778617c76e2,2017-03-26,69,2017-03-26,Sunday,0,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,35.0
252106,air_a17f0778617c76e2,2017-03-20,31,2017-03-20,Monday,1,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,3.0


In [6]:
df.select_dtypes(include = np.object).columns.tolist()

/var/folders/ry/lx4fw4hj4c58xy781z3_7cbm0000gn/T/ipykernel_24812/1461041111.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df.select_dtypes(include = np.object).columns.tolist()


['id', 'calendar_date', 'day_of_week', 'genre', 'area']

In [7]:
df['yesterday'] = df.groupby('id').apply(lambda x: x['visitors'].shift()).values
df['last_week'] = df.groupby('id').apply(lambda x: x['visitors'].shift(7)).values

# fill in missing reservations
df['reserve_visitors'] = df['reserve_visitors'].fillna(0)

# drop missing values from shifts
df = df.dropna()

In [8]:
X = df[['id', 'yesterday', 'last_week', 'reserve_visitors', 'day_of_week', 'holiday', 'area']]
y = df['visitors']

In [9]:
# let's assume these were our optimized parameters
tree = DecisionTreeRegressor(max_depth = 7, max_features = 0.8, min_samples_leaf = 10)

pipe = make_pipeline(ce.TargetEncoder(), tree)

In [10]:
# fit the tree, and export it
pipe.fit(X, y)

Pipeline(steps=[('targetencoder',
                 TargetEncoder(cols=['id', 'day_of_week', 'area'])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(max_depth=7, max_features=0.8,
                                       min_samples_leaf=10))])

In [11]:
# the pickle module allows you to export saved models
import pickle

# rb -- WRITE the file in BYTES
with open('pipe.pkl', 'wb') as export:
    # this creates an external version of the file that we can now import later on
    pickle.dump(pipe, export)

In [12]:
# we can now import this, and re-use it on new data:  very handy

with open('pipe.pkl', 'rb') as import_:
    # this will import the pickled object again
    pipe2 = pickle.load(import_)

In [13]:
# here it is
pipe2

Pipeline(steps=[('targetencoder',
                 TargetEncoder(cols=['id', 'day_of_week', 'area'])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(max_depth=7, max_features=0.8,
                                       min_samples_leaf=10))])

In [14]:
# and we can use it to make new predictions
pipe2.predict(X)

array([19.55975395, 30.4109589 , 31.14644351, ..., 54.23311132,
       41.44688186, 54.23311132])